In [3]:
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score,confusion_matrix
import numpy as np
import pandas as pd

In [8]:
train = pd.read_csv('C:/Users/doong/Desktop/PythonWorkspace/운송량예측공모전/train(전처리 끝).csv', encoding='utf-8')
test = pd.read_csv('C:/Users/doong/Desktop/PythonWorkspace/운송량예측공모전/test(전처리 끝).csv', encoding='utf-8')
ss = pd.read_csv('C:/Users/doong/Desktop/공모전/운송량 예측/235867_물류 유통량 예측 경진대회/sample_submission.csv', encoding='utf-8')

In [9]:
train.drop(["Unnamed: 0", "index"], axis=1, inplace=True)
test.drop(["Unnamed: 0",'index'], axis=1, inplace=True)

In [6]:
# train["INVC_CONT"] = np.log1p(train["INVC_CONT"])

In [10]:
train.describe()

,SEND_SPG_INNB,REC_SPG_INNB,INVC_CONT,DL_GD_LCLS_NM_0,DL_GD_LCLS_NM_1,DL_GD_LCLS_NM_2,DL_GD_LCLS_NM_3,DL_GD_LCLS_NM_4,DL_GD_LCLS_NM_5,DL_GD_MCLS_NM_0,...,DL_GD_MCLS_NM_10,DL_GD_MCLS_NM_11,DL_GD_MCLS_NM_12,DL_GD_MCLS_NM_13,DL_GD_MCLS_NM_14,DL_GD_MCLS_NM_15,DL_GD_MCLS_NM_16,DL_GD_MCLS_NM_17,DL_GD_MCLS_NM_18,DL_GD_MCLS_NM_19
count,32000.000000,32000.000000,32000.000000,32000.000000,32000.000000,32000.000000,32000.000000,32000.000000,32000.000000,32000.000000,...,32000.000000,32000.000000,32000.000000,32000.000000,32000.000000,32000.000000,32000.000000,32000.000000,32000.000000,32000.000000
mean,2344.954656,14502.947219,4.767875,0.007531,0.063125,0.866812,0.037250,0.018187,0.007094,0.028844,...,0.037250,0.011187,0.010031,0.010719,0.029094,0.007094,0.035187,0.019844,0.010250,0.013062
std,908.476216,8279.118860,5.752122,0.086457,0.243191,0.339783,0.189377,0.133631,0.083926,0.167370,...,0.189377,0.105179,0.099654,0.102977,0.168072,0.083926,0.184256,0.139465,0.100724,0.113544
min,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1867.000000,7250.750000,3.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2560.000000,14626.500000,3.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3151.000000,22146.250000,5.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3683.000000,27825.000000,239.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [15]:
train.columns

Index(['SEND_SPG_INNB', 'REC_SPG_INNB', 'INVC_CONT', 'DL_GD_LCLS_NM_0',
       'DL_GD_LCLS_NM_1', 'DL_GD_LCLS_NM_2', 'DL_GD_LCLS_NM_3',
       'DL_GD_LCLS_NM_4', 'DL_GD_LCLS_NM_5', 'DL_GD_MCLS_NM_0',
       'DL_GD_MCLS_NM_1', 'DL_GD_MCLS_NM_2', 'DL_GD_MCLS_NM_3',
       'DL_GD_MCLS_NM_4', 'DL_GD_MCLS_NM_5', 'DL_GD_MCLS_NM_6',
       'DL_GD_MCLS_NM_7', 'DL_GD_MCLS_NM_8', 'DL_GD_MCLS_NM_9',
       'DL_GD_MCLS_NM_10', 'DL_GD_MCLS_NM_11', 'DL_GD_MCLS_NM_12',
       'DL_GD_MCLS_NM_13', 'DL_GD_MCLS_NM_14', 'DL_GD_MCLS_NM_15',
       'DL_GD_MCLS_NM_16', 'DL_GD_MCLS_NM_17', 'DL_GD_MCLS_NM_18',
       'DL_GD_MCLS_NM_19'],
      dtype='object')

In [11]:
# 대 + 중
X = train.iloc[:,3:]
y = train["INVC_CONT"]

In [37]:
# # 중
# X = train.iloc[:,9:]
# y = train["INVC_CONT"]

In [64]:
# # 배송 + 수송 + 대 + 중
# X = train.drop("INVC_CONT", axis=1)
# y = train["INVC_CONT"]

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X, y ,test_size=0.2)

In [67]:
# def lgb_evaluate(numLeaves, maxDepth, scaleWeight, minChildWeight, subsample, colSam):
#     reg=lgb.LGBMRegressor(num_leaves=31, max_depth= 2,scale_pos_weight= scaleWeight, min_child_weight= minChildWeight, 
#                           subsample= 0.4, colsample_bytree= 0.4, learning_rate=0.05,   n_estimators=20)
#     scores = cross_val_score(reg, X_train, y_train, cv=5, scoring='roc_auc')
#     scores = cross_val_score(reg, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
#     return np.mean(np.sqrt(-scores))

In [68]:
# def bayesOpt(train_x, train_y):
#     lgbBO = BayesianOptimization(lgb_evaluate, {  'numLeaves':  (5, 90),  'maxDepth': (2, 90),   'scaleWeight': (1, 10000),  
#                                                 'minChildWeight': (0.01, 70), 'subsample': (0.4, 1), 'colSam': (0.4, 1) })
#     lgbBO.minimize(init_points=5, n_iter=50)

In [69]:
# bayesOpt(X_train, y_train)

In [16]:
cat_features = ['DL_GD_LCLS_NM_0',
       'DL_GD_LCLS_NM_1', 'DL_GD_LCLS_NM_2', 'DL_GD_LCLS_NM_3',
       'DL_GD_LCLS_NM_4', 'DL_GD_LCLS_NM_5', 'DL_GD_MCLS_NM_0',
       'DL_GD_MCLS_NM_1', 'DL_GD_MCLS_NM_2', 'DL_GD_MCLS_NM_3',
       'DL_GD_MCLS_NM_4', 'DL_GD_MCLS_NM_5', 'DL_GD_MCLS_NM_6',
       'DL_GD_MCLS_NM_7', 'DL_GD_MCLS_NM_8', 'DL_GD_MCLS_NM_9',
       'DL_GD_MCLS_NM_10', 'DL_GD_MCLS_NM_11', 'DL_GD_MCLS_NM_12',
       'DL_GD_MCLS_NM_13', 'DL_GD_MCLS_NM_14', 'DL_GD_MCLS_NM_15',
       'DL_GD_MCLS_NM_16', 'DL_GD_MCLS_NM_17', 'DL_GD_MCLS_NM_18',
       'DL_GD_MCLS_NM_19']

In [17]:
def bayes_parameter_opt_lgb(X, y, init_round=20, opt_round=10, n_folds=5, random_seed=0, n_estimators=1000, learning_rate=0.001, output_process=False):
    train_data = lgb.Dataset(data = X_train, label=y_train, categorical_feature = list(X.columns),free_raw_data=False)
    
    def lgb_eval(num_leaves, feature_fraction, max_depth , min_split_gain, min_child_weight):
            params = {
            "objective" : "regression", "bagging_fraction" : 0.8, "bagging_freq": 1,
            "min_child_samples": 20, "reg_alpha": 1, "reg_lambda": 1,"boosting": "dart",
            "learning_rate" : 0.001, "subsample" : 0.8, "colsample_bytree" : 0.8, "verbosity": -1, "metric" : 'rmse'
        }
    
            params['feature_fraction'] = max(min(feature_fraction, 1), 0)
            params['max_depth'] = int(round(max_depth))
            params['num_leaves'] = int(round(num_leaves))
            params['min_split_gain'] = min_split_gain
            params['min_child_weight'] = min_child_weight
            cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, verbose_eval =200,stratified=False)
            return (-1.0 * np.array(cv_result['rmse-mean'])).max()
        
    lgbBO = BayesianOptimization(lgb_eval, {'feature_fraction': (0.5, 0.9),
                                            'max_depth': (0, 10),
                                            'num_leaves' : (200,250),
                                            'min_split_gain': (0.001, 0.1),
                                            'min_child_weight': (20, 50)}, random_state=0)
        
    lgbBO.maximize(init_points=init_round, n_iter=opt_round,acq='ei')

        # output optimization process
    if output_process==True: lgbBO.points_to_csv("bayes_opt_result.csv")

    target_list = []
    for result in lgbBO.res:
        target = result['target']
        target_list.append(target)
    max_dict = lgbBO.res[np.argmax(np.array(target_list))]
    print(max_dict)

In [18]:
opt_params = bayes_parameter_opt_lgb(X_train, y_train, init_round=30, n_folds=5, random_seed=0, n_estimators=1000, learning_rate=0.001)

TypeError: bayes_parameter_opt_lgb() got an unexpected keyword argument 'categorical_feature'

In [13]:
model = lgb.LGBMRegressor(random_state=2021, n_estimators=1000, bagging_fraction=0.88, learning_rate=0.001,
                         max_depth=8, subsample=0.7, feature_fraction=0.8914473368931056, boosting_type='dart',
                         reg_lambda=5, n_jobs=-1, min_child_weight=33.84438086758796, min_split_gain=0.07827238845235909, num_leaves=205)

In [14]:
model.fit(X_train,y_train, 
          eval_set = [(X_test,y_test)],
          eval_metric = 'rmse',
          early_stopping_rounds=10,
          verbose=10
         )

C:\Users\doong\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\doong\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\doong\anaconda3\lib\site-packages\lightgbm\callback.py:223: UserWarning: Early stopping is not available in dart mode
  _log_warning('Early stopping is not available in dart mode')


[10]	valid_0's rmse: 0.467418	valid_0's l2: 0.21848
[20]	valid_0's rmse: 0.543651	valid_0's l2: 0.295556
[30]	valid_0's rmse: 0.542853	valid_0's l2: 0.29469
[40]	valid_0's rmse: 1.03089	valid_0's l2: 1.06272
[50]	valid_0's rmse: 1.02832	valid_0's l2: 1.05745
[60]	valid_0's rmse: 1.0255	valid_0's l2: 1.05166
[70]	valid_0's rmse: 1.09089	valid_0's l2: 1.19005
[80]	valid_0's rmse: 1.08734	valid_0's l2: 1.18231
[90]	valid_0's rmse: 1.08332	valid_0's l2: 1.17358
[100]	valid_0's rmse: 1.08033	valid_0's l2: 1.16712
[110]	valid_0's rmse: 1.07607	valid_0's l2: 1.15793
[120]	valid_0's rmse: 1.07324	valid_0's l2: 1.15184
[130]	valid_0's rmse: 1.07324	valid_0's l2: 1.15184
[140]	valid_0's rmse: 1.0706	valid_0's l2: 1.14618
[150]	valid_0's rmse: 1.06936	valid_0's l2: 1.14353
[160]	valid_0's rmse: 1.07016	valid_0's l2: 1.14523
[170]	valid_0's rmse: 1.06506	valid_0's l2: 1.13435
[180]	valid_0's rmse: 1.08156	valid_0's l2: 1.16978
[190]	valid_0's rmse: 1.0944	valid_0's l2: 1.1977
[200]	valid_0's rmse:

LGBMRegressor(bagging_fraction=0.88, boosting_type='dart',
              feature_fraction=0.8914473368931056, learning_rate=0.001,
              max_depth=8, min_child_weight=33.84438086758796,
              min_split_gain=0.07827238845235909, n_estimators=1000,
              num_leaves=205, random_state=2021, reg_lambda=5, subsample=0.7)

In [17]:
test_X = test.iloc[:,2:]
pred = model.predict(test_X)
pred = np.exp(pred)
ss["INVC_CONT"] = pred

In [18]:
ss

,index,INVC_CONT
0,32000,1.565027
1,32001,1.565027
2,32002,1.565027
3,32003,1.565027
4,32004,1.565027
...,...,...
4635,36635,1.565027
4636,36636,1.565027
4637,36637,1.565027
4638,36638,1.565027


In [64]:
ss.to_csv('sample_submssion.csv',index = False)